---
title: "Persistent Images of Gravitionally Lensed Galaxies"
bibliography: bibliography.bib
title-slide-attributes:
    data-background-image: https://euclid.caltech.edu/download/AvmImage/58/binary/jpg_original

format: html
    
---




# Introduction
Gravitationally lensed galaxies are an active area of research in astronomy as investigating their properties can give insights into gravitational


## Astronomy
- What is a gravitationally lensed galaxy?
- Why do we care about them?

## Persistent Homology
- What is it?
- How is it helpful?
- How has it been used in astronomy before?
- What do I plan 

## Machine Learning
- What is machine learning?
- Why do we need/want to perform clustering?
- How do the clustering algorithms work and what are they?

# Data
The Euclid Space telescope was launched in ___ with the intention of mapping 2/3 of the night sky. On March 19, 2025, the Euclid Collaboration released their Quick Data Release (Q1).

- What data do I have? What does it look like?
- Where did I get the data from?
- Who funded the data?  - ESA
- When was the data collected?
- Why was the data collected?
- How do I plan on using the data?

The code chunk below imports the necessary modules for working with this data.


In [ ]:
from astroquery.esa.euclid import Euclid
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, ImageNormalize, PercentileInterval, AsinhStretch, LogStretch
import gtda.plotting
import pandas as pd
from astropy.io import ascii
import glob
import numpy as np
import os
from gtda.images import Binarizer
from gtda.images import RadialFiltration
from gtda.homology import CubicalPersistence
from gtda.diagrams import Scaler
from gtda.diagrams import PersistenceImage
from persim import PersImage
from persim import PersistenceImager

Once all the libraries are imported, we can extract the data. Each lensed galaxy was found via a machine learning classification model, thus there existed a distribution of objects with associated probabilities that they were lensed galaxies, based on the model. Currently, this project only incorporates objects that had a "grade" of "A" (i.e. were of the best quality).


In [ ]:
# get coordinates from list of targets (only select best images)
all_targets = ascii.read("D:/data/targets/q1_discovery_engine_lens_catalog.csv")
mask = (all_targets["subset"] == "discovery_engine") & (all_targets["grade"] == "A")
targets = all_targets[mask]

The target list only contains the celestial coordinates of the grade-A lensed galaxies. Luckily, we can download cutouts of the targets. Note, this code chunk is intentially set to not run as all of the cutouts have already been extracted. Note, we have ~250 targets.


In [ ]:
# | eval: false
# don't run unless need redownload images!!!
# download the images for each target
empty_images = 0
for i in range(len(targets)):
    r, d = targets["right_ascension"][i], targets["declination"][i]
    coord = SkyCoord(r, d, unit=u.deg, frame='icrs')
    radius = u.Quantity(0.5, u.deg)

    print(f"r: {r}")
    print(f"d: {d}")
    print(coord)
    print(radius)

    # search euclid's mosaic product catalog for targets
    job = Euclid.cone_search(coordinate=coord, radius=radius, table_name="sedm.mosaic_product", ra_column_name="ra", dec_column_name="dec", columns="*", async_job=True, verbose=True)
    cone_results = job.get_results()
    example_file = cone_results[cone_results['instrument_name'] == 'VIS'][0]
    print(f"cone_results: {cone_results}")

    # save results to output path
    file_path = example_file["file_path"] + "/" + example_file["file_name"]
    instrument = example_file["instrument_name"]
    obs_id = example_file["tile_index"]
    radius = 0.1 * u.arcmin
    output_folder = 'D:/data/'

    print(f"file_path: {file_path}")
    print(f"instrument: {instrument}")
    print(f"obs_id: {obs_id}")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_file=output_folder + f'cutouts_0.1arcmin/target_{r}_{d}.fits'
    saved_cutout_filepath = Euclid.get_cutout(file_path=file_path,
        instrument=instrument, id=obs_id, 
        coordinate=coord, radius=radius, output_file=output_file)

    print("Cutout saved at", output_file)

    # looking at the cutout we made
    try:
        hdul = fits.open(output_file)
        print(fits.info(output_file))
        image_data = hdul[0].data
        plt.imshow(image_data, interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(image_data, interval=PercentileInterval(99.5),
                            stretch=AsinhStretch()))
        plt.colorbar()
        plt.show()
    except:
        empty_images += 1

print(empty_images)

Once the cutouts are extracted, we can display them so get a sense of what the data actually looks like. The code chunk below shows an example of what a gravitationally lensed galaxy looks like from the Euclid space telescope.


In [ ]:
# get data from each fits image
fits_files_clean = glob.glob("D:/data/cutouts_0.1arcmin_119x119/*.fits")

data_list = []
for file in fits_files_clean:
    hdul = fits.open(file)
    data = hdul[0].data
    data_list.append(data)

# Analysis
Once we have the images, the next step is to compute the persistent homology for each image. The code chunk below displays the example image using Python, a Giotto-tda heatmap mehtod, and calculates the cubical persistence of the example image.



## FITS Image


In [ ]:
# load in the image
plt.imshow(data_list[0].reshape((119, 119)), interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(data_list[0], interval=PercentileInterval(99.5), stretch=AsinhStretch()))
plt.colorbar()
plt.show()

## Preprocessing


In [ ]:
# rescaling to [0,1]
scale = 10
data_list = [data.reshape((1,119,119)) for data in data_list]  # reshape for cubical complex
data_list = [scale * (data - data.min()) / (data.max() - data.min()) for data in data_list]  # rescale
data_invert_list = [scale * np.ones(data.shape) - data for data in data_list]

In [ ]:
# display preprocessed image
plt.imshow(data_invert_list[0].reshape(119,119), interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(data_invert_list[0], interval=PercentileInterval(99.5), stretch=AsinhStretch()))
plt.colorbar()
plt.show()

## Calculate Persistence


In [ ]:
cubical_persistence = CubicalPersistence(homology_dimensions=(0,1), n_jobs=-1, infinity_values=scale)

print("calculating cubical persistence (h0) on inverted images...")
cubical_invert_data_list = []
for inverted_data in data_invert_list:
    cubical_invert_data = cubical_persistence.fit_transform(inverted_data)
    cubical_invert_data_list.append(cubical_invert_data)

print("calculating cubical persistence (h1) on images...")
cubical_data_list = []
for data in data_list:
    cubical_data = cubical_persistence.fit_transform(data)
    cubical_data_list.append(cubical_data)

In [ ]:
cubical_diagram0_list = []
for cubical_invert_data in cubical_invert_data_list:
    mask0 = cubical_invert_data[:,:,2] == 0  # id h0 in inverted data
    data_cubical0 = cubical_invert_data[mask0]  # select h0 inverted data
    cubical_diagram0 = np.delete(data_cubical0, 2, 1)  # remove the h0 identifier
    cubical_diagram0_list.append(cubical_diagram0)  # add to list

cubical_diagram1_list = []
for cubical_data in cubical_data_list:
    mask1 = cubical_data[:,:,2] == 1  # id h1 in inverted data
    data_cubical1 = cubical_data[mask1]  # select h1 inverted data
    cubical_diagram1 = np.delete(data_cubical1, 2, 1)  # remove the h1 identifier
    cubical_diagram1_list.append(cubical_diagram1)  # add to list

## Create Persistence Images


In [ ]:
# fit h0 images
h0_pimgr = PersistenceImager(pixel_size=1)
h0_pimgr.weight_params = {"n": 1.5}  # tuning parameter (maybe [1, 2])
h0_pimgr.fit(cubical_diagram0_list)
h0_pimgr.birth_range = (0, scale)
h0_pimgr.pers_range = (0, scale)
h0_imgs = h0_pimgr.transform(cubical_diagram0_list, skew=True)

# fit h1 images
h1_pimgr = PersistenceImager(pixel_size=1)
h1_pimgr.weight_params = {"n": 1.5}  # tuning parameter (maybe [1, 2])
h1_pimgr.fit(cubical_diagram1_list)
h1_pimgr.birth_range = (0, scale)
h1_pimgr.pers_range = (0, scale)
h1_imgs = h1_pimgr.transform(cubical_diagram1_list, skew=True)

In [ ]:
def persistence_diagram(bdh_array, ax): 
    h0mask = bdh_array[:,:,2] == 0
    h1mask = bdh_array[:,:,2] == 1
    h0birth = bdh_array[h0mask][:,0] 
    h0death = bdh_array[h0mask][:,1]
    h1birth = bdh_array[h1mask][:,0] 
    h1death = bdh_array[h1mask][:,1]

    ax.scatter(h0birth, h0death, alpha=0.5, marker='.', label="h0")
    ax.scatter(h1birth, h1death, alpha=0.5, marker='.', label="h1")
    ax.set_xlabel("Birth")
    ax.set_ylabel("Death")
    ax.legend(loc="lower right")
    oldxlim = ax.get_xlim()
    oldylim = ax.get_ylim()
    ax.axline((0,0), (scale + 1,scale + 1), linestyle='--', c="grey")
    ax.set_xlim(-0.5, scale + 0.5)
    ax.set_ylim(-0.5, scale + 0.5)
    return


def pipeline_plot(index: str = 0, return_fig: bool = False):
    fig, ax = plt.subplots(2,3,figsize=(10,8))
    # inverted image
    im_invert = ax[0][0].imshow(data_invert_list[index].reshape((119, 119)), interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(data_invert_list[index], interval=PercentileInterval(99.5), stretch=AsinhStretch()))
    ax[0][0].set_title("Raw Image (Inverted)")
    fig.colorbar(im_invert, ax=ax[0][0])

    # persistence diagram (inverted)
    persistence_diagram(cubical_invert_data_list[index], ax[0][1])
    ax[0][1].set_title("H0 Persistence Diagram (Inverted Data)")

    # persistence image (inverted)
    h1_pimgr.plot_image(h1_imgs[index], ax=ax[0][2])
    ax[0][2].set_title("H0 Persistence Image (Inverted Data)")

    # raw image
    im = ax[1][0].imshow(data_list[index].reshape((119, 119)), interpolation='nearest', cmap='gray', origin='lower', norm=ImageNormalize(data_list[index], interval=PercentileInterval(99.5), stretch=AsinhStretch()))
    ax[1][0].set_title("Raw Image")
    fig.colorbar(im, ax=ax[1][0])

    # persistence diagram (normal)
    persistence_diagram(cubical_data_list[index], ax[1][1])
    ax[1][1].set_title("H1 Persistence Diagram")

    # persistence image (normal)
    h0_pimgr.plot_image(h0_imgs[index], ax=ax[1][2])
    ax[1][2].set_title("H1 Persistence Image")

    # adjust subplot images
    fig.subplots_adjust(hspace=0.4, wspace=0.3)

    if return_fig:
        return fig, ax
    
    return

In [ ]:
pipeline_plot(-12)

In [ ]:
# convert to vector
h0_vec = [image.reshape(100) for image in h0_imgs]
h1_vec = [image.reshape(100) for image in h1_imgs]

final_vec = []
for i in range(len(h0_vec)):
    combined = np.concatenate((h0_vec[i], h1_vec[i]), axis=None)
    final_vec.append(combined)

## Convert Original Images to Vectors and Save Both Vectors to a CSV 


In [ ]:
fits_files = glob.glob("D:/data/cutouts_0.1arcmin/*.fits")
names = [i.split("\\")[-1].replace('.fits', "") for i in fits_files]

final_table = pd.DataFrame(np.vstack(final_vec), index=names)
final_table.to_csv("D:/results/persistent_results.csv")

fits_image_vec = [image.reshape(119 * 119) for image in data_list]
final_fits_table = pd.DataFrame(np.vstack(fits_image_vec), index=names)
final_fits_table.to_csv("D:/results/fits_results.csv")

## Clustering

Hierarchical clustering.


# Results and Discussion


## Cluster: Natural Images {.center .nonincremental}

![](cluster_natural.png){.fragment width="1200" height="600"}

## Cluster: Persistence Images {.center .nonincremental}

![](cluster_persistent.png){.fragment width="1600" height="600"}


## {.center}

![](cluster_persistent.png){.fragment width="1600" height="600"}

# Conclusions

# References
blah blah [@Chen2015, @Sousbie2011]